# Comparing Performance of Vanilla and Convolutional Neural Networks

**Time**
- Teaching: 15 minutes

**Questions**
- "How do vanilla neural networks compare to convolutional neural networks?"

* * * * *

The neural networks we have created so far are known as *vanilla neural networks*. 

These have many great usecases, but for problems in computer vision, we often use a different architecture called covolutional neural networks.

We will review the the details of how the work in our slides tomorrow, but for today, let's just compare their efficacy to vanilla neural nets!

In [2]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from tensorflow.keras.utils import to_categorical

In [ ]:
def get_mnist_data(subset=True):
    """
    Returns the MNIST dataset as a tuple:
    (x_train, y_train, x_val, y_val, x_test, y_test)
    
    When subset=TRUE:
    Returns only a subset of the mnist dataset.
    Especially important to use if you are on datahub and only have 1-2GB of memory.
    """
    
    if subset:
        N_TRAIN = 5000
        N_VALIDATION = 1000
        N_TEST = 1000
    else:
        N_TRAIN = 48000
        N_VALIDATION = 12000
        N_TEST = 10000
    
    (x_train_and_val, y_train_and_val), (x_test, y_test) = mnist.load_data()
    
    x_train = x_train_and_val[:N_TRAIN,:,:]
    y_train = y_train_and_val[:N_TRAIN]
    
    x_val = x_train_and_val[N_TRAIN: N_TRAIN + N_VALIDATION,:,:]
    y_val = y_train_and_val[N_TRAIN: N_TRAIN + N_VALIDATION]
    
    x_test = x_test[:N_TEST]
    y_test = y_test[:N_TEST]
    
    return x_train, y_train, x_val, y_val, x_test, y_test
    
    

In [ ]:
convnet = Sequential()

convnet.add(Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))
convnet.add(MaxPooling2D((2, 2)))
convnet.add(Conv2D(64, (3, 3), activation='relu'))
convnet.add(MaxPooling2D((2, 2)))
convnet.add(Conv2D(64, (3, 3), activation='relu'))

convnet.add(Flatten())
convnet.add(Dense(64, activation='relu'))
convnet.add(Dense(10, activation='softmax'))

convnet.compile(optimizer='rmsprop',
                loss='categorical_crossentropy',
                metrics=['accuracy'])

Note that convolutional neural networks don't need our data flattened, in fact they work well on multidimensional input data!

In [ ]:
convnet = convnet.fit(x_train_2d,
                      y_train_trans, 
                      epochs=10,
                      batch_size=128,
                      bvalidation_data=(x_val_2d, y_val_trans))

In [ ]:
def plot_epoch_accuracy(history_dict):
    """
    Plots the training and validation accuracy of a neural network.
    """
    
    acc = history_dict['accuracy']
    val_acc = history_dict['val_accuracy']
    epochs = range(1, len(acc) + 1)
    plt.plot(epochs, acc, color = 'navy', alpha = 0.8, label='Training Accuracy')
    plt.plot(epochs, val_acc, color = 'green', label='Validation Accuracy')
    plt.title('Training and validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    return plt.show()

In [ ]:
plot_epoch_accuracy(history_convnet.history)

So how well does our CNN perform on the test set?

In [ ]:
get_model_accuracy(convnet,x_test_2d, y_test_trans)

In [ ]:
plot_wrong_predictions(convnet, x_test_2d, y_test_trans)